***
<div style="text-align:center">
     <h1 align="center">
         <i class="fas fa-calendar-day" aria-hidden="true">  Day 16 of #3D_vision_journey </i>
    </h1>      
</div>

<h3 style='text-align: center;'>
    <img  src="../images/mediapipe.png" height="50"/>   +   <img  src="../images/open3d_logo.png" height="50"/>     
</h3>

* Load the libraries

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import imutils
import open3d
import open3d as o3d
import copy, time

draw_bit = 1 # draw it or not!

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
INFO - 2022-05-01 10:27:17,390 - utils - NumExpr defaulting to 8 threads.


###  access two sub-modules from mediapipe
- The `drawing_utils` sub-module exposes a function that allows to `draw the detected face landmarks` over the image.

    * `thickness`: Thickness for drawing the annotation. Defaults to 2 pixels.
    * `circle_radius`: Circle radius for drawing the landmarks. Defaults to 2 pixels.
    * `color`: Color for drawing the annotation. Defaults to green. 

    
- The `face_mesh`     sub-module exposes the function necessary to `do the face detection and landmarks estimation`.
    * `static_image_mode`: Boolean indicating if the images it processes should be treated as 
        * `unrelated images` (True) 
        * or as a `video stream` (False).
        
    * `min_detection_confidence`. 
        Minimum confidence value (in an interval between 0.0 and 1.0) from the face detection model for the detection to be considered successful [1]. Defaults to 0.5,        

    * `min_tracking_confidence`: 
        Minimum confidence value (in an interval from 0.0 to 1.0) from the landmark-tracking model for the face landmarks to be considered tracked successfully, or otherwise face detection will be invoked automatically on the next input image [1] . Passing a higher value can increase robustness at the cost of higher latency. This parameter is `ignored` if the `static_image_mode parameter` is set to True [1]. Defaults to 0.5.

In [2]:
mp_face_mesh      = mp.solutions.face_mesh
mp_drawing        = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

#drawing_spec      = mp_drawing.DrawingSpec  (thickness=1, circle_radius=1)
circleDrawingSpec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1, color=(0,255,0))
lineDrawingSpec   = mp_drawing.DrawingSpec(thickness=1, color=(0,255,0))
face_mesh         = mp_face_mesh.FaceMesh(static_image_mode=False, 
                                            max_num_faces=1,
                                            refine_landmarks=True, 
                                            min_detection_confidence=0.5)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### read the video frame and do face mesh detection

In [12]:
def get_landmarks(frame, pcd, draw_mesh=False):
    frame = imutils.resize(frame, width=400)    

    image_rows, image_cols = frame.shape[:2]    

    

    annotated_image = frame.copy()

    # we expect just one detection 
    # Convert the BGR image to RGB before processing.
    results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    #if not results.multi_face_landmarks:
    #    continue
    face_landmarks = results.multi_face_landmarks[0]
    
    idx_to_coordinates = {}
    landmark_xyz = []
    for idx, landmark in enumerate(face_landmarks.landmark):
        if landmark.visibility < 0 or landmark.presence < 0:
            continue
        
        # pixel coordinates landmark
        landmark_px = mp_drawing._normalized_to_pixel_coordinates(landmark.x, landmark.y, image_cols, image_rows)
        landmark_xyz.append([landmark.x, landmark.y,landmark.z])
        #landmark_xyz.append([landmark_px[0], landmark_px[0], int((0.5 + landmark.z) * 5)])
        
        if landmark_px and not draw_mesh:
            idx_to_coordinates[idx] = landmark_px
            cv2.circle(annotated_image, landmark_px, int((0.5 + landmark.z) * 5), (255, 0, 0), -1)
            
        if draw_mesh:
            mp_drawing.draw_landmarks(
                    image=annotated_image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_tesselation_style())
            
            mp_drawing.draw_landmarks(
                    image=annotated_image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_contours_style())
                        
            mp_drawing.draw_landmarks(
                    image=annotated_image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_iris_connections_style()) 

    xyz = np.asarray(landmark_xyz)
    pcd.points = o3d.utility.Vector3dVector(xyz)
    R = pcd.get_rotation_matrix_from_xyz((0, np.pi, np.pi))              
    pcd.rotate(R, center=(0, 0, 0))    
    
    return pcd, annotated_image 



video_url = "../data/videos/face_move.MOV"
cam = cv2.VideoCapture(video_url)
#file = open('landmarks.txt', 'w')

# we expect just one detection 
ret, frame = cam.read()
pcd = o3d.geometry.PointCloud()
pcd, annotated_image = get_landmarks(frame, pcd, draw_mesh=False)

vis = o3d.visualization.Visualizer()
vis.create_window(width=640, height=480)
vis.add_geometry(pcd)

while True:
    ret, frame = cam.read()
    
    if not ret:
        break
    
    pcd, annotated_image = get_landmarks(frame, pcd, draw_mesh=False)    
    
    cv2.imshow('Output', annotated_image)
    if cv2.waitKey(1) == ord('q'):
        break
    
    vis.update_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()
    


    time.sleep(0.05)
    
    
vis.destroy_window()    
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x55ac3f09d5a0) is not the object's thread (0x55ac40557050).
Cannot move to target thread (0x55ac3f09d5a0)

QObject::moveToThread: Current thread (0x55ac3f09d5a0) is not the object's thread (0x55ac40557050).
Cannot move to target thread (0x55ac3f09d5a0)

QObject::moveToThread: Current thread (0x55ac3f09d5a0) is not the object's thread (0x55ac40557050).
Cannot move to target thread (0x55ac3f09d5a0)

QObject::moveToThread: Current thread (0x55ac3f09d5a0) is not the object's thread (0x55ac40557050).
Cannot move to target thread (0x55ac3f09d5a0)

QObject::moveToThread: Current thread (0x55ac3f09d5a0) is not the object's thread (0x55ac40557050).
Cannot move to target thread (0x55ac3f09d5a0)

QObject::moveToThread: Current thread (0x55ac3f09d5a0) is not the object's thread (0x55ac40557050).
Cannot move to target thread (0x55ac3f09d5a0)

QObject::moveToThread: Current thread (0x55ac3f09d5a0) is not the object's thread (0x55ac40557050).
Cannot move to tar